In [ ]:
import pandas as pd
import numpy as np
import datetime

## 要输入的地方是lotid，读取表格

In [ ]:
L = 'L2E9221A9AA'
dfm = pd.read_excel('./%s/M_%s.xls' % (L, L))
dfcm = pd.read_excel('./%s/MC_%s.xls' % (L, L))
dfo = pd.read_excel('./%s/O_%s.xls' % (L, L))
print(dfm.shape)
print(dfcm.shape)
print(dfo.shape)

# 全部的处理过程

## 1.要注意有的Lot有到AUU的记录，有些没有，所以要稍作处理  
## 2.目前我们canceltki也算作制程时间，所以处理过程更简单了

In [ ]:
L = 'L2E9212A2AA'

dfm = pd.read_excel('./%s/M_%s.xls' % (L, L))
dfcm = pd.read_excel('./%s/MC_%s.xls' % (L, L))
dfo = pd.read_excel('./%s/O_%s.xls' % (L, L))
print(dfm.shape)
print(dfcm.shape)
print(dfo.shape)

Lotid = dfm['LOTID'][0]

# 有的LOT比较复杂，这里的开始时间要单独处理

dfcm['TIME'] = pd.to_datetime(dfcm['TIME'])
AUU_start = dfcm[(dfcm['DEST MACHINE'] == '2AUU01') & (dfcm['STATE'] == 'CREATED')]['TIME']
print('%s 进入Array时间为 %s 。' % (Lotid ,AUU_start.iloc[0]))

#####################################################################
# 这个是对于没有到AUU记录的单独做的处理，直接选用dfm的第一条记录时间作为开始时间

# dfm['TIME'] = pd.to_datetime(dfm['TIME'])
# AUU_start = dfm.iloc[0][0]
# print('%s 进入Array时间为 %s 。' % (Lotid ,AUU_start))


dfm = dfm[['TIME', 'TRANSPORTJOBID', 'STATE']]     #只选取我们需要用到的三列'TIME', 'TRANSPORTJOBID', 'STATE'
print('删除前 %s ' % len(dfm))
l = dfm['TRANSPORTJOBID']
dfmcount = pd.DataFrame([l.value_counts()]).T      # 统计每个TRANSPORTJOBID出现的次数
list_one = dfmcount[dfmcount['TRANSPORTJOBID'] == 1].index.tolist()  # 如果出现次数是1，记录下来下一步准备删除
print('准备删除 %s 条记录' % len(list_one))
dfm=dfm[-dfm.TRANSPORTJOBID.isin(list_one)]         # 删除TRANSPORTJOBID是唯一的记录
print('删除后 %s ' % len(dfm))

# 上面是删除唯一的，这里是删除 3 个的情况，目前只遇到3个，如果大于三个下次再判断
# 如果是3个，就删除掉重复的最后一条，可能是两个complete或者termination
while True:
    dfm_three = pd.DataFrame(dfm.TRANSPORTJOBID.value_counts())
    if not dfm_three[dfm_three['TRANSPORTJOBID'] == 3].index.tolist():
        print('没有3个的记录了，继续执行')
        break 
    job_three = dfm_three[dfm_three['TRANSPORTJOBID'] == 3].index[0]
    three_end = dfm[dfm['TRANSPORTJOBID'] == job_three].index[-1]
    dfm = dfm[~dfm.index.isin([three_end])]
    print('处理一个重复记录')


dfm_time = pd.DataFrame()  # 新建一个表格用来处理搬送时间
# 原表中‘STATE’是‘CREATED’的那些行的时间提取出来
dfm_time['created'] = dfm[dfm['STATE'] == 'CREATED']['TIME'].tolist()
# 原表中‘STATE’不是‘CREATED’（也就是‘COMPLETED’或者是‘TERMINATED’）的那些行的时间提取出来
dfm_time['ended'] = dfm[dfm['STATE'] != 'CREATED']['TIME'].tolist()
# 转换成时间格式，为下一步计算差值以及总时间做准备
dfm_time = dfm_time.astype(np.datetime64)
# 计算每次搬送的时间，并求和
dfm_time['delta'] = dfm_time['ended'] - dfm_time['created']
trantime = dfm_time['delta'].sum()
print('搬送次数是：%s' % (len(dfm_time)+1))
print('%s 在 Array 搬送总时间为： %s' % (Lotid, trantime))

dfo = dfo[['EventTime', 'EventName', 'EQP ID']]                 # 只提取'EventTime', 'EventName'后续处理
dfo['EventTime'] = pd.to_datetime(dfo['EventTime'])             # 转换时间戳，便于后面的计算
shipidx = int(dfo[dfo['EventName'] == 'Ship'].index.values)
dfo = dfo.iloc[shipidx:]
ship_time = dfo[dfo['EventName'] == 'Ship']['EventTime']

sumtime = (ship_time.iloc[0] - AUU_start.iloc[0])
#############################################################################
# sumtime = (ship_time.iloc[0] - AUU_start)

print('%s 在 Array 总的时间为：%s ' % (Lotid, sumtime))


###############################################################################
# 这一块是之前处理canceltki的函数，现在不需要了，
# dfo.index = range(len(dfo))
# cancel = dfo[dfo['EventName'] == 'CancelTrackIn']
# print('开始处理被cancel的trackin ')
# print(cancel)
# cancel_list = list(cancel.index)
# cancel_tki_list = [x+1 for x in cancel_list]
# dfo = dfo[~dfo.index.isin(cancel_tki_list)]
# print('被cancel的trackin处理完毕')

#################################################################################
# 现在的做法是一个 tki 对应 一个tko 或者是 canceltki，实现思路等同于上面的处理搬送时间
dfo = dfo[dfo['EventName'].isin(['TrackIn', 'TrackOut', 'CancelTrackIn'])]
dfo_time = pd.DataFrame()
dfo_time['tki'] = dfo[dfo['EventName'] == 'TrackIn']['EventTime'].tolist()
print('trackin的次数为 %s' % len(dfo_time['tki']))
dfo_time['tko'] = dfo[dfo['EventName'] != 'TrackIn']['EventTime'].tolist()
print('trackout/canceltrackin的次数为 %s' % len(dfo_time['tko']))
dfo_time = dfo_time.astype(np.datetime64)
dfo_time['delta'] = dfo_time['tko'] - dfo_time['tki']
runtime = dfo_time['delta'].sum()
print('%s 在 Array 总的制程时间为：%s' % (Lotid, runtime))

####################################################################################
# 这是原来的处理
# dfo_time = pd.DataFrame()
# dfo_time['tki'] = dfo[dfo['EventName'] == 'TrackIn']['EventTime'].tolist()
# dfo_time['tko'] = dfo[dfo['EventName'] == 'TrackOut']['EventTime'].tolist()
# dfo_time = dfo_time.astype(np.datetime64)
# dfo_time['delta'] = dfo_time['tko'] - dfo_time['tki']
# runtime = dfo_time['delta'].sum()
# print('%s 在 Array 总的制程时间为 ： %s' % (Lotid, runtime))

waittime = (sumtime - trantime - runtime)
print('%s 在 Array 总的等待时间为：% s' % (Lotid, waittime))

## 下面格式输出，稍微美观一点

In [ ]:
print('本批lot为： %s ' % Lotid)
print('搬送次数是：%s' % (len(dfm_time)+1))
for x in [sumtime, trantime, runtime, waittime]:
    d = x.days
    h = (x.seconds//3600) + d*24
    m = (x.seconds//60)%60
    s = x.seconds - ((x.seconds//3600))*3600 - ((x.seconds//60)%60)*60
    print(x, "          %s 小时 %s 分 %s 秒" % (h, m, s))

In [ ]:
# print('本批lot为： %s ' % Lotid)
# print('搬送次数是：%s' % len(dfm_time))
# print('%s 在 Array 总的时间为:       %s' % (Lotid, sumtime))
# print('%s 在 Array 搬送总时间为:     %s' % (Lotid, trantime))
# print('%s 在 Array 总的制程时间为:   %s' % (Lotid, runtime))
# print('%s 在 Array 总的等待时间为:   %s' % (Lotid, waittime))

## 输出到csv

In [ ]:
df_lot = pd.DataFrame()
df_lot['过程'] = ['%s 在 Array 总的时间为' % Lotid, '%s 在 Array 搬送总时间为' % Lotid, '%s 在 Array 总的制程时间为' % Lotid, '%s 在 Array 总的等待时间为' % Lotid]
df_lot['时间'] = [sumtime, trantime, runtime, waittime]

topath = './%s/res_%s.csv' % (L, L)
df_lot.to_csv(topath, encoding='utf_8_sig', index=False)
print('成功保存到res_%s.csv文件!' % Lotid)

# 转换成秒，绘制饼图

In [ ]:
s_runtime = runtime.total_seconds()
s_trantime = trantime.total_seconds()
s_waittime = waittime.total_seconds()
# print(s_runtime, s_trantime, s_waittime)

from pyecharts import Pie

attr = ["制程时间", "搬送时间", "等待时间"]
s_time = [s_runtime, s_trantime, s_waittime]
pie = Pie(('%s在Array总时长%s' % (Lotid, sumtime)), width=900, height=500,  title_pos='left', title_text_size=21)
pie.add("", attr, s_time, 
        is_label_show=True,
        radius=[45, 70], 
        legend_pos = 'right',
        legend_orient = 'vertical',
        label_text_size=19, legend_text_size=17)

# 绘制在各个站点的runtime/ Notruntime

In [ ]:
dfo_every = dfo.sort_values(['EventTime'])
dfo_every = dfo_every.reset_index(drop=True)
indexs = dfo_every[dfo_every['EventName'] == 'TrackIn']['EQP ID'].tolist()

dfo_every_run = pd.DataFrame()
dfo_every_run['tki'] = dfo_every[dfo_every['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_every_run['tko'] = dfo_every[dfo_every['EventName'] != 'TrackIn']['EventTime'].tolist()
dfo_every_run = dfo_every_run.astype(np.datetime64)
dfo_every_run['rtime'] = dfo_every_run['tko'] - dfo_every_run['tki']
r_time = list((dfo_every_run['rtime'].values).astype('timedelta64[m]'))

dfo_every_nrun = dfo_every
dfo_every_nrun = dfo_every_nrun.drop([0, len(dfo_every_nrun)-1])
dfo_every_Nrun = pd.DataFrame()
dfo_every_Nrun['tki'] = dfo_every_nrun[dfo_every_nrun['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_every_Nrun['tko'] = dfo_every_nrun[dfo_every_nrun['EventName'] != 'TrackIn']['EventTime'].tolist()
dfo_every_Nrun = dfo_every_Nrun.astype(np.datetime64)
dfo_every_Nrun['nrtime'] = dfo_every_Nrun['tki'] - dfo_every_Nrun['tko']
nr_time = list((dfo_every_Nrun['nrtime'].values).astype('timedelta64[m]'))
nr_time.append(0)
nr_time = [0-x for x in nr_time]

from pyecharts import  Bar

attr = indexs
v1 = r_time
v2 = nr_time

bar = Bar(Lotid, width=1000, height=600)
bar.add("runtime(单位：min)", attr, v1,  
        mark_line=['average'], 
        mark_point=["min", 'max'], 
        is_more_utils=True, 
        xaxis_rotate=50,
        is_datazoom_show=True,
        datazoom_type='both',)
bar.add("Not_runtime(单位：min)", attr, v2,
        mark_line=['average'], 
        mark_point=["min", 'max'], )

## 画各个shop的runtime图的分析步骤如下所示

In [ ]:
dfo_every = dfo.sort_values(['EventTime'])
dfo_every = dfo_every.reset_index(drop=True)
dfo_every

In [ ]:
indexs = dfo_every[dfo_every['EventName'] == 'TrackIn']['EQP ID'].tolist()
print(len(indexs), indexs)

In [ ]:
dfo_every_run = pd.DataFrame()
dfo_every_run['tki'] = dfo_every[dfo_every['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_every_run['tko'] = dfo_every[dfo_every['EventName'] != 'TrackIn']['EventTime'].tolist()
dfo_every_run = dfo_every_run.astype(np.datetime64)
dfo_every_run['rtime'] = dfo_every_run['tko'] - dfo_every_run['tki']
dfo_every_run

In [ ]:
r_time = list((dfo_every_run['rtime'].values).astype('timedelta64[m]'))
r_time

In [ ]:
dfo_every_nrun = dfo_every
dfo_every_nrun = dfo_every_nrun.drop([0, len(dfo_every_nrun)-1])
dfo_every_nrun

In [ ]:
dfo_every_Nrun = pd.DataFrame()
dfo_every_Nrun['tki'] = dfo_every_nrun[dfo_every_nrun['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_every_Nrun['tko'] = dfo_every_nrun[dfo_every_nrun['EventName'] != 'TrackIn']['EventTime'].tolist()
dfo_every_Nrun = dfo_every_Nrun.astype(np.datetime64)
dfo_every_Nrun['nrtime'] = dfo_every_Nrun['tki'] - dfo_every_Nrun['tko']
dfo_every_Nrun

In [ ]:
nr_time = [0-x for x in nr_time]

In [ ]:
nr_time = list((dfo_every_Nrun['nrtime'].values).astype('timedelta64[m]'))
nr_time.append(0)
nr_time = [0-x for x in nr_time]

In [ ]:
from pyecharts import  Bar

attr = indexs
v1 = r_time
v2 = nr_time

bar = Bar(Lotid, width=1000, height=600)
bar.add("runtime(单位：min)", attr, v1,  
        mark_line=['average'], 
        mark_point=["average"], 
        is_more_utils=True, 
        xaxis_rotate=50,
        is_datazoom_show=True,
        datazoom_type='both',)
bar.add("Not_runtime(单位：min)", attr, v2,)